# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [3]:
import json
import boto3
import panel as pn  # GUI

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [4]:
def get_completion(prompt, modelId="ai21.jamba-1-5-mini-v1:0"):
    messages = [{"role": "user", "content": prompt}]
    body = {
        "messages": messages,
        "temperature": 0
    }
    response = bedrock_runtime.invoke_model(
        body=json.dumps(body),
        modelId=modelId
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get("choices")[0]["message"]["content"]


## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [5]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

 Hola, me gustaría pedir una licuadora.


In [6]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

 The language of the sentence "Combien coûte le lampadaire?" is French.


In [7]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

 Sure, here are the translations:

**French:**
"Je veux commander un ballon de basket."

**Spanish:**
"Quiero pedir una pelota de baloncesto."

**English pirate:**
"Ahoy, I be wantin' to order a basketball, matey!"


In [8]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

 Sure! Here are the translations for "Would you like to order a pillow?" in both formal and informal Spanish:

**Formal:**

* "¿Le gustaría pedir una almohada?"

**Informal:**

* "¿Te gustaría pedir una almohada?"


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [9]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [10]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message ( The language of the text is French.): La performance du système est plus lente que d'habitude.
 **English:** The system performance is slower than usual.

**Korean:** 시스템 성능이 보통보다 느립니다. 

Original message ( The language of the text is **Spanish**. 

Here's a breakdown:

* **Mi monitor:** My monitor
* **tiene:** has
* **píxeles:** pixels
* **que no se iluminan:** that do not light up 


Let me know if you have any other text snippets you'd like me to identify!
): Mi monitor tiene píxeles que no se iluminan.
 Here is the translation of the text to English and Korean:

**English:**
"My monitor has pixels that do not light up."

**Korean:**
"내 모니터에는 불이 들어오지 않는 픽셀이 있습니다." 

Original message ( The language is Italian. 

The phrase "Il mio mouse non funziona" translates to "My mouse doesn't work" in English. 

): Il mio mouse non funziona
 Here is the translation of "Il mio mouse non funziona" to English and Korean:

**English:** My mouse doesn't work.

**Korean:** 내 마우스가 작

## Try it yourself!
Try some translations on your own!

## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [11]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

 Certainly! Here's a more formal version of the message:


---

Dear [Recipient's Name],

I hope this message finds you well. My name is Joe, and I am writing to provide you with the specifications for a standing lamp that I believe may be of interest to you.

Please find the details attached for your review.

Best regards,

Joe [Your Last Name]


---

Feel free to adjust the details as needed.


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [12]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

 ```html
<h2>Restaurant Employees</h2>
<table>
  <thead>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>
```

**Explanation:**

1. **Title:** We use an `<h2>` tag to display the title "Restaurant Employees".
2. **Table Structure:**
   - `<table>` tag creates the table.
   - `<thead>` tag defines the table header.
   - `<tr>` tag represents a table row.
   - `<th>` tag defines a table header cell. We use it for "Name" and "Email".
3. **Table Body:**
   - `<tbody>` tag defines the table body.
   - Each employee's data is enclosed within a `<tr>` tag.
   - `<td>` tag represents a table data cell, used for displaying the name and email.

**Key Points:**

- This HTML code assumes the Python dictionary is directly accessible in

In [13]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [14]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

 No errors found.
 No errors found.
 No errors found.
 No errors found.
 No errors found.
 No errors found.
 No errors found.

Corrected version:

```
This phrase is to check chatGPT for spelling ability.```


In [15]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

 Here's the proofread and corrected version of your review:


---

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, which I don't think was designed to be asymmetrical. It's a bit small for what I paid for it, though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter.


---

Let me know if you need any further adjustments!


In [ ]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

In [ ]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

## Try it yourself!
Try changing the instructions to form your own review.

Thanks to the following sites:

https://writingprompts.com/bad-grammar-examples/